In [1]:
import datetime
import ee
import hydrafloods as hf

ee.Initialize()

In [2]:
from IPython.display import Image

In [7]:
# define region
region = ee.Geometry.Point(105,12).buffer(100000).bounds(maxError=1)
# define time period
startTime = datetime.datetime(2019,1,1)
endTime = startTime + datetime.timedelta(31)

In [8]:
viirs = hf.collection.Viirs(region,startTime,endTime)

In [9]:
print(f"Number of images: {viirs.nImages}")

Number of images: 32


In [10]:
thumbnail = ee.Image(viirs.collection.sort('system:time_start',False).first()).getThumbUrl(
    {'region':region,'min':50,'max':5500,'bands':'swir2,nir,green','gamma':1.5,
     'format':'png','dimensions':512,'crs':'epsg:4326'}
)
# display result
Image(url = thumbnail)

In [11]:
landsat = hf.collection.Landsat8(region,startTime,endTime)

In [12]:
print(landsat.nImages)

15


In [13]:
thumbnail = ee.Image(landsat.collection.mean()).getThumbUrl(
    {'region':region,'min':50,'max':5500,'bands':'swir2,nir,green','gamma':1.5,
     'format':'png','dimensions':512,'crs':'epsg:4326'}
)
# display result
Image(url = thumbnail)

In [14]:
fused = viirs.apply(hf.fusion.starfm,fineCollection=landsat,targetDate=endTime.strftime('%Y-%m-%d'))

In [15]:
fused.nImages

6

In [16]:
thumbnail = ee.Image(fused.collection.sort('system:time_start',False).first()).getThumbUrl(
    {'region':region,'min':50,'max':5500,'bands':'swir2,nir,green','gamma':1.5,
     'format':'png','dimensions':512,'crs':'epsg:4326'}
)
# display result
Image(url = thumbnail)

In [17]:
downscaledWater = fused.apply(hf.thresholding.bmaxOtsu,initialThreshold=-16,band='mndwi')


Not going to try to display results as it will end with a `Computation timed out.`

In [19]:
hf.geeutils.batchExport(downscaledWater.collection,'users/kelmarkert/hydrafloods_examples',
                        prefix='viirs_fused',suffix='bmax',scale=10,
                        metadata={'sensor':'viirs_fused','algorithm':'bmax'},
                        pyramiding={'.default':'mode'},
                        verbose=True)

running export for viirs_fused_20190201_bmax
running export for viirs_fused_20190131_bmax
running export for viirs_fused_20190130_bmax
running export for viirs_fused_20190129_bmax
running export for viirs_fused_20190128_bmax
running export for viirs_fused_20190127_bmax


In [21]:
ee.batch.Task.list()[:downscaledWater.nImages]

[<Task EXPORT_IMAGE: viirs_fused_20190127_bmax (READY)>,
 <Task EXPORT_IMAGE: viirs_fused_20190128_bmax (READY)>,
 <Task EXPORT_IMAGE: viirs_fused_20190129_bmax (READY)>,
 <Task EXPORT_IMAGE: viirs_fused_20190130_bmax (READY)>,
 <Task EXPORT_IMAGE: viirs_fused_20190131_bmax (READY)>,
 <Task EXPORT_IMAGE: viirs_fused_20190201_bmax (RUNNING)>]